### Split Dataset

In [1]:
import pandas as pd
import random
import math


In [2]:

train_set_df = pd.read_csv('D:/allforwork/AI/nnsp/NS-2025-02-data/train_set.csv')

origin_dataset = [[] for i in range(2)]
siz = len(train_set_df)

for i in range(len(train_set_df)):
    text = train_set_df['review'].iloc[i]
    label = train_set_df['label'].iloc[i]
    origin_dataset[label].append(text)


#### Smaller Dataset

In [3]:
n_p = 0.08

small_origin_dataset = [[],[]]
for i in range(2):
    print(len(origin_dataset[i]))
    n_size = math.floor(n_p * len(origin_dataset[i]))
    small_origin_dataset[i] = random.sample(origin_dataset[i], k=n_size)

origin_dataset = small_origin_dataset
print(len(origin_dataset[0]) + len(origin_dataset[1]))

54073
53916
8638


In [4]:
val_p = 0.15

train_dataset = []
val_dataset = []



for i in range(2):
    subsiz = math.ceil(val_p * len(origin_dataset[i]))
    val_data = random.sample(origin_dataset[i], k=subsiz)
    for j in range(len(val_data)):
        val_dataset.append({'text': val_data[j], 'label': i})

    for j in range(len(origin_dataset[i])):
        if origin_dataset[i][j] not in val_data:
            train_dataset.append({'text': origin_dataset[i][j], 'label': i})


In [5]:

random.shuffle(train_dataset)
random.shuffle(val_dataset)

train_dataset_df = pd.DataFrame(train_dataset)
val_dataset_df = pd.DataFrame(val_dataset)


In [6]:

train_dataset_df.to_csv('D:/allforwork/AI/nnsp/NS-2025-02-data/8k_train_df.csv', index=False)
val_dataset_df.to_csv('D:/allforwork/AI/nnsp/NS-2025-02-data/8k_val_df.csv', index=False)

### Laod Data

In [7]:
from datasets import load_dataset


d:\Anaconda\envs\sklearn-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

dataset = load_dataset("csv", data_files='D:/allforwork/AI/nnsp/NS-2025-02-data/train_set.csv')
dataset = dataset['train'].train_test_split(train_size=0.8, seed=42)
dataset

In [8]:
dataset = load_dataset("csv", data_files={'train':'D:/allforwork/AI/nnsp/NS-2025-02-data/8k_train_df.csv',
                                          'test':'D:/allforwork/AI/nnsp/NS-2025-02-data/8k_val_df.csv'})
dataset

Generating train split: 7342 examples [00:00, 332953.97 examples/s]
Generating test split: 1296 examples [00:00, 123502.02 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7342
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1296
    })
})

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, AutoModelForSequenceClassification

model_name = "D:/allforwork/AI/nnsp/Qwen/Qwen2_5-0_5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_mode=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.config.pad_token = tokenizer.pad_token
model.config.pad_token_id = tokenizer.pad_token_id
import torch
print(model)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at D:/allforwork/AI/nnsp/Qwen/Qwen2_5-0_5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Qwen2ForSequenceClassification(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Q

In [10]:
from peft import LoraConfig, TaskType, get_peft_model
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.05)

In [11]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 542,464 || all params: 494,577,024 || trainable%: 0.1097


In [12]:
def preprocess_function(examples):
    # 添加任务指令（关键！）
    prompt = [f"【任务说明】\n你是一个专业的网络内容审核助手,结合下列手册和你的专业素养,判断后面给出的网络评论是否包含辱骂、歧视、仇恨等情绪,正常评论输出0,负面评论输出1\n【手册】\n1.后面给出的网络评论格式为微博评论格式,形如:当前用户评论内容//@其他用户A:A的评论内容//@其他用户B:B的评论内容...\n2.请结合其他用户评论内容作为上文来帮助你判断当前用户评论内容是否违规\n3.评论中包含表情包，格式为[表情内容]，如：[泪]\n4.请注意分辨反讽、玩笑、报道等容易混淆的内容\n【待判断网络评论】: {text}\n 答案(0/1)：" for text in examples['text']]
    inputs = tokenizer(
        prompt, 
        truncation=True, 
        max_length=512, 
        padding="max_length",
        return_tensors="pt"
    )
    inputs["labels"] = examples["label"]
    return inputs


In [13]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, batch_size=4, remove_columns=dataset['train'].column_names)

Map: 100%|██████████| 1296/1296 [00:00<00:00, 2608.13 examples/s]


In [14]:
print(tokenized_dataset.column_names)

{'train': ['input_ids', 'attention_mask', 'labels'], 'test': ['input_ids', 'attention_mask', 'labels']}


In [15]:
training_args = TrainingArguments(
    output_dir="D:/allforwork/AI/nnsp/NS-02-model/v1",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="D:/allforwork/AI/nnsp/NS-02-model/logs",  # 日志目录
    logging_strategy="steps",
    logging_steps=50,     # 每50步记录一次
    report_to="none",     # 禁用第三方服务
    eval_strategy="steps",
    eval_steps=200,       # 每200步验证一次
    save_strategy="steps",
    save_steps=400,
    # fp16=True,
    # gradient_accumulation_steps=4,
    num_train_epochs=2,
    # 启用进度条增强
    disable_tqdm=False,          # 必须开启
    dataloader_pin_memory=True,  # 加速数据加载
    label_names=["labels"]
)

In [16]:
print(tokenizer.pad_token)
print(model.config.pad_token)
model.config.pad_token_id = tokenizer.pad_token_type_id

<|endoftext|>
<|endoftext|>


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

trainer.train()

C:\Users\47328\AppData\Local\Temp\ipykernel_10304\2122121990.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
200,0.669900,0.684391
400,0.655700,0.629727
600,0.535000,0.507052
800,0.263200,0.293369
1000,0.117500,0.160442
1200,0.128200,0.140344
1400,0.185200,0.114439
1600,0.152500,0.090166
1800,0.061200,0.087964


TrainOutput(global_step=1836, training_loss=0.3454196584250672, metrics={'train_runtime': 36825.8366, 'train_samples_per_second': 0.399, 'train_steps_per_second': 0.05, 'total_flos': 1.6169065671819264e+16, 'train_loss': 0.3454196584250672, 'epoch': 2.0})

In [19]:
import peft

In [20]:
save_name = 'Qwen2_5-0_5B-0.08size-v1'
trainer.save_model(f"D:/allforwork/AI/nnsp/NS-02-model/{save_name}")
tokenizer.save_pretrained(f"D:/allforwork/AI/nnsp/NS-02-model/{save_name}")
if isinstance(model, peft.PeftModel):
    model.save_pretrained(f"D:/allforwork/AI/nnsp/NS-02-model/{save_name}/lora_adapters")
    model.base_model.save_pretrained(f"D:/allforwork/AI/nnsp/NS-02-model/{save_name}/base_model")

In [21]:
test_set_df = pd.read_csv("D:/allforwork/AI/nnsp/NS-2025-02-data/test_set.csv")

test_dataset = []

for i in range(len(test_set_df)):
    test_dataset.append(f"【任务说明】\n你是一个专业的网络内容审核助手,结合下列手册和你的专业素养,判断后面给出的网络评论是否包含辱骂、歧视、仇恨等情绪,正常评论输出0,负面评论输出1\n【手册】\n1.后面给出的网络评论格式为微博评论格式,形如:当前用户评论内容//@其他用户A:A的评论内容//@其他用户B:B的评论内容...\n2.请结合其他用户评论内容作为上文来帮助你判断当前用户评论内容是否违规\n3.评论中包含表情包，格式为[表情内容]，如：[泪]\n4.请注意分辨反讽、玩笑、报道等容易混淆的内容\n【待判断网络评论】: {test_set_df['review'].iloc[i]}\n 答案(0/1)：")

In [22]:
test_dataset[:5]

['【任务说明】\n你是一个专业的网络内容审核助手,结合下列手册和你的专业素养,判断后面给出的网络评论是否包含辱骂、歧视、仇恨等情绪,正常评论输出0,负面评论输出1\n【手册】\n1.后面给出的网络评论格式为微博评论格式,形如:当前用户评论内容//@其他用户A:A的评论内容//@其他用户B:B的评论内容...\n2.请结合其他用户评论内容作为上文来帮助你判断当前用户评论内容是否违规\n3.评论中包含表情包，格式为[表情内容]，如：[泪]\n4.请注意分辨反讽、玩笑、报道等容易混淆的内容\n【待判断网络评论】: 回复@沂蒙山区老母鸡:如何解决保鲜问题？ //@沂蒙山区老母鸡:我想办法给您快递好吗？[太开心] //@中青旅新疆公司郑穗:我明天就试，问题是哪里去找真正的 “老母鸡”？现在的鸡，差不多都是几个月出笼。  //@北京律师成义:\n 答案(0/1)：',
 '【任务说明】\n你是一个专业的网络内容审核助手,结合下列手册和你的专业素养,判断后面给出的网络评论是否包含辱骂、歧视、仇恨等情绪,正常评论输出0,负面评论输出1\n【手册】\n1.后面给出的网络评论格式为微博评论格式,形如:当前用户评论内容//@其他用户A:A的评论内容//@其他用户B:B的评论内容...\n2.请结合其他用户评论内容作为上文来帮助你判断当前用户评论内容是否违规\n3.评论中包含表情包，格式为[表情内容]，如：[泪]\n4.请注意分辨反讽、玩笑、报道等容易混淆的内容\n【待判断网络评论】: 幻想的吧，大中午的。 //@小树林s:这牛逼吹的有点大了。瑞银的老板都脸红。[哈哈]\n 答案(0/1)：',
 '【任务说明】\n你是一个专业的网络内容审核助手,结合下列手册和你的专业素养,判断后面给出的网络评论是否包含辱骂、歧视、仇恨等情绪,正常评论输出0,负面评论输出1\n【手册】\n1.后面给出的网络评论格式为微博评论格式,形如:当前用户评论内容//@其他用户A:A的评论内容//@其他用户B:B的评论内容...\n2.请结合其他用户评论内容作为上文来帮助你判断当前用户评论内容是否违规\n3.评论中包含表情包，格式为[表情内容]，如：[泪]\n4.请注意分辨反讽、玩笑、报道等容易混淆的内容\n【待判断网络评论】: #实时旅讯# 厦门鼓浪屿昨日出现海市蜃楼！[太开心] 海上升仙山，美哭了！新年刚开始就

In [29]:
model_name = "D:/allforwork/AI/nnsp/NS-02-model/v1/checkpoint-1600"
infer_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_mode=True)
infer_model = AutoModelForSequenceClassification.from_pretrained(model_name)
infer_model.config.pad_token = infer_tokenizer.pad_token
infer_model.config.pad_token_id = infer_tokenizer.pad_token_id

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at D:/allforwork/AI/nnsp/Qwen/Qwen2_5-0_5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
from transformers import pipeline

infer_pipeline = pipeline(
    'text-classification',
    model=infer_model,
    tokenizer=infer_tokenizer,
    device=0
)

results = infer_pipeline(test_dataset, batch_size=4)

Device set to use cuda:0


In [31]:
len(results)

11999

In [32]:
results[:15]

[{'label': 'LABEL_0', 'score': 0.9944257736206055},
 {'label': 'LABEL_1', 'score': 0.9045822024345398},
 {'label': 'LABEL_1', 'score': 0.9283919334411621},
 {'label': 'LABEL_1', 'score': 0.6515127420425415},
 {'label': 'LABEL_1', 'score': 0.9342349767684937},
 {'label': 'LABEL_1', 'score': 0.9800474047660828},
 {'label': 'LABEL_1', 'score': 0.9818713665008545},
 {'label': 'LABEL_1', 'score': 0.8558772802352905},
 {'label': 'LABEL_0', 'score': 0.6429874300956726},
 {'label': 'LABEL_1', 'score': 0.7952083945274353},
 {'label': 'LABEL_1', 'score': 0.7754880785942078},
 {'label': 'LABEL_1', 'score': 0.9683642387390137},
 {'label': 'LABEL_1', 'score': 0.6028019785881042},
 {'label': 'LABEL_1', 'score': 0.8764522075653076},
 {'label': 'LABEL_1', 'score': 0.9472465515136719}]

In [33]:
output_result = [1 if res['label'] == 'LABEL_1' else 0 for res in results ]

In [34]:
output_result[:15]

[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1]

In [35]:
import json
ans = {"results": []}
for t, res in enumerate(output_result):
    ans["results"].append({"id":t + 1,"label":res})

with open("D:/allforwork/AI/nnsp/NS-2025-02-data/results.json", mode="w") as f:
    json.dump(ans, f, indent=2)